In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import requests
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
master_dataset = pd.read_csv('master_dataset_final.csv')
print(master_dataset.shape)

(2500, 4)


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,2), min_df=2)
countMatrix = vectorizer.fit_transform(master_dataset['soup'])

In [12]:
countMatrix.shape

(2500, 6908)

In [17]:
master_dataset

,release_date,title,main_director,soup
0,2015-06-17,Minions,Kyle Balda,assist aftercreditssting duringcreditssting ev...
1,2017-05-30,Wonder Woman,Patty Jenkins,dccomic hero greekmytholog island worldwari su...
2,2017-03-16,Beauty and the Beast,Bill Condon,franc magic castl fairytal music curs anthropo...
3,2017-06-28,Baby Driver,Edgar Wright,robberi atlanta music crimeboss romanc tinnitu...
4,2014-10-24,Big Hero 6,Chris Williams,brotherbrotherrelationship hero talent reveng ...
...,...,...,...,...
2495,2002-01-23,The Count of Monte Cristo,Kevin Reynolds,lossoflov lover(female) ex-lov tortur napoleon...
2496,2014-09-04,Boychoir,François Girard,talent music school sing orphan dustinhoffman ...
2497,2007-03-16,Dead Silence,James Wan,detect clown tombston funer legend ventriloqui...
2498,2014-09-06,Black or White,Mike Binder,grievingwidow widow kevincostner jillianestell...


In [18]:
cos_sim=cosine_similarity(countMatrix)
cos_sim_df = pd.DataFrame(cos_sim, index=master_dataset.index, columns=master_dataset.index)
cos_sim_df

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,1.000000,0.000000,0.036860,0.000000,0.080257,0.173494,0.112351,0.033826,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,0.000000,1.000000,0.000000,0.000000,0.043033,0.186052,0.120483,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,0.036860,0.000000,1.000000,0.036860,0.034021,0.000000,0.000000,0.028677,0.000000,0.0,...,0.0,0.0,0.0,0.079057,0.0,0.0,0.066815,0.064550,0.000000,0.000000
3,0.000000,0.000000,0.036860,1.000000,0.000000,0.000000,0.000000,0.033826,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.078811,0.000000,0.000000,0.039406
4,0.080257,0.043033,0.034021,0.000000,1.000000,0.053376,0.034565,0.031220,0.038490,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.072739,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000
2496,0.000000,0.000000,0.066815,0.078811,0.072739,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.000000
2497,0.000000,0.000000,0.064550,0.000000,0.000000,0.000000,0.000000,0.000000,0.036515,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.105409,0.000000
2498,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.115470,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.105409,1.000000,0.000000


In [25]:
def recommendations(title):
    try:
        idx = master_dataset[master_dataset['title'] == title].index[0]
    except IndexError:
        print("Title not found.")
        return
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    for i in sim_scores:
        print(master_dataset.iloc[i[0]]['title'], master_dataset.iloc[i[0]]['main_director'], master_dataset.iloc[i[0]]['release_date'])


In [26]:
recommendations('Minions')

Despicable Me 3 Kyle Balda 2017-06-15
Harold & Kumar Escape from Guantanamo Bay Jon Hurwitz 2008-04-25
MacGruber Jorma Taccone 2010-05-21
Ant-Man Peyton Reed 2015-07-14
Couples Retreat Peter Billingsley 2009-09-19
